In [1]:
#Install and load packages
import malariagen_data
import os
import numpy as np
import pandas as pd
import allel
import xarray as xr
import glob

/home/harunnn/.conda/envs/gaard/lib/python3.10/site-packages/anjl/_canonical.py:187: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  n_threads = get_num_threads()


In [2]:
ag3 = malariagen_data.Ag3(pre = True)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release_master_us_central1/
Data releases available : 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 3.10, 3.11, 3.12, 3.13, 3.14
Results cache           : None
Cohorts analysis        : 20250131
AIM analysis            : 20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 14.0.0
Client location         : England, United Kingdom
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v14.0.0/Ag3.html

In [3]:

#function to extract biallelic genotypes
#sample_sets= ["1288-VO-UG-DONNELLY-VMF00168","1288-VO-UG-DONNELLY-VMF00219"]
sample_query= ("aim_species == 'gambiae'")

def extract_and_filter_snps(region, maf_threshold, output_filename ):
    
    array_snps = ag3.snp_calls(region=region,
                               #sample_sets=sample_sets,
                               sample_query=sample_query,
                               site_mask='gamb_colu' )
    
    gt = allel.GenotypeArray(array_snps['call_genotype'])
    
    no_missing = gt.count_missing(1) == 0
    gt_freq=gt.count_alleles().to_frequencies()
    which_pos = (np.max(gt_freq,1) < (1 - maf_threshold)) & no_missing
    gt_filtered = gt[which_pos,:]
   
    gt_biallelic = np.sum(gt_filtered>0,2)
    #convert to dataframe
    df_gt = pd.DataFrame(gt_biallelic)
    pos = array_snps['variant_position'][which_pos]
    
    chrom = np.array(array_snps.contigs)[array_snps['variant_contig']][which_pos]
    #snp_id = np.apply_along_axis(':'.join, 0, [chrom, pos.astype('str')])
    snp_id = np.apply_along_axis(lambda x: np.asarray(':'.join(x), dtype = 'object'), 0, [chrom, pos.values.astype('str')])
    df_gt.set_index(snp_id, inplace = True)
    df_gt.columns=array_snps.sample_id
    # Save DataFrame to CSV file
    df_gt.to_csv(output_filename)

    return(df_gt)
    



In [4]:
output_directory ='/home/harunnn/lstm_scratch/network_scratch/llineup/llineup-genomics/llineup_publication/scripts_notebooks/haplotype_clustering'

region= '2R:28470000-28500000'  #added 10Kb upstreeam and downstream of the focal haplotype cluster region

# Dictionary comprehension to call the function for each region
gt =  extract_and_filter_snps(region, 0.02, os.path.join(output_directory, f'gt_{region}.csv')) 


In [5]:
ug_hap_calls = (
    ag3.haplotypes(region = region, 
                   analysis = 'gamb_colu', 
                   #sample_sets = sample_sets, 
                   sample_query = sample_query)
    .set_index(variants = 'variant_position')
)




Access haplotypes: ⠋ (0:00:00.00)

In [17]:
genotype_snps = gt.index.str.split(':').str[1]
genotype_snps

Index(['2300012', '2300046', '2300840', '2300851', '2301093', '2302099',
       '2309921', '2321129', '2321468', '2323414',
       ...
       '2902325', '2902356', '2902518', '2902616', '2902649', '2902675',
       '2902688', '2902763', '2903041', '2903107'],
      dtype='object', length=1592)

In [18]:
ug_hap_calls.variants.values

array([2300003, 2300004, 2300007, ..., 2903267, 2903271, 2903273],
      dtype=int32)

In [9]:
gt

,VBS50275-6645STDY11194051,VBS50276-6645STDY11194052,VBS50283-6645STDY11194053,VBS50286-6645STDY11194055,VBS50287-6645STDY11194056,VBS50288-6645STDY11194057,VBS50289-6645STDY11194058,VBS50291-6645STDY11194059,VBS50293-6645STDY11194060,VBS50294-6645STDY11194061,...,VBS67010-6645STDY13057654,VBS67012-6645STDY13057656,VBS67013-6645STDY13057657,VBS67014-6645STDY13057658,VBS67020-6645STDY13057664,VBS67021-6645STDY13057665,VBS67022-6645STDY13057666,VBS67024-6645STDY13057668,VBS67025-6645STDY13057669,VBS67026-6645STDY13057670
2L:2300012,1,1,2,1,0,0,1,0,1,1,...,2,0,0,1,0,1,1,0,0,1
2L:2300046,1,1,2,1,0,0,1,0,1,1,...,2,0,0,1,0,1,1,0,0,1
2L:2300840,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,1,0,0,1,1,1
2L:2300851,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,1,0,0,1,1,1
2L:2301093,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2L:2902675,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2L:2902688,1,0,0,1,2,2,1,2,1,1,...,0,1,1,0,1,1,1,1,2,0
2L:2902763,2,2,2,2,2,2,2,2,2,2,...,2,1,1,2,1,2,2,1,2,1
2L:2903041,2,2,2,2,2,2,2,2,2,2,...,2,1,0,2,0,2,2,1,1,1


In [6]:
# Some SNPs don't have haplotype calls. 
# Identify the SNPs for which there are haplotype data, and those for which there aren't
genotype_snps = gt.index.str.split(':').str[1].astype(int)

snps_with_hap_data = [x for x in genotype_snps if x in ug_hap_calls.variants.values]
#snps_no_hap_data = [x for x in ug_snp_calls['variants'] if x not in ug_hap_calls.variants.values]

# pull out the genotype calls where we haplotypes calls
gt_trimmed = gt.loc[gt.index.str.split(':').str[1].astype(int).isin(snps_with_hap_data)]
output_file = os.path.join(output_directory, f'gt_trimmed_{region}.csv')
gt_trimmed.to_csv(output_file)
gt_trimmed

,VBS00343-4651STDY7017197,VBS00345-4651STDY7017199,VBS00350-4651STDY7017200,VBS00352-4651STDY7017202,VBS00353-4651STDY7017203,VBS01052-4651STDY7017307,VBS01054-4651STDY7017309,VBS01056-4651STDY7017311,VBS01061-4651STDY7017314,VBS01070-4651STDY7017318,...,SAMN02899221,SAMN02899222,SAMN02899223,SAMN02899224,SAMN02899225,SAMN02899226,SAMN02899227,SAMN02899228,SAMN02899229,SAMN02899230
2R:28470429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2R:28470695,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2R:28470827,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,0,0,0,0
2R:28470829,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2R:28470841,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2R:28499716,1,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2R:28499733,1,0,0,2,1,1,1,2,2,1,...,0,0,0,0,0,0,0,1,0,0
2R:28499795,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2R:28499954,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
snp_set = set(snps_with_hap_data)  # Convert list to set for faster lookup
if 2422651 in snp_set:
    print("snp is in the list.")
else:
    print("snp is not in the list.")

snp is not in the list.
